# Import

In [1]:
from __future__ import print_function, division

In [2]:
import sys,os
#os.environ['KMP_DUPLICATE_LIB_OK']='True' # uncomment this line if omp error occurs on OSX for python 3
os.environ['OMP_NUM_THREADS']='1' # set number of OpenMP threads to run in parallel
os.environ['MKL_NUM_THREADS']='1' # set number of MKL threads to run in parallel

In [3]:
quspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,quspin_path)

In [4]:
from quspin.operators import hamiltonian # Hamiltonians and operators
from quspin.basis import boson_basis_1d # Hilbert space spin basis_1d
from quspin.basis.user import user_basis # Hilbert space user basis
from quspin.basis.user import next_state_sig_32,op_sig_32,map_sig_32,count_particles_sig_32 # user basis data types signatures
from numba import carray,cfunc # numba helper functions
from numba import uint32,int32,float64 # numba data types
import numpy as np
from scipy.special import comb

# Parameters

In [5]:
N=4 # lattice sites
sps=3 # states per site
Np=4 # total number of bosons

# Create boson user basis object 

In [6]:
######  function to call when applying operators
@cfunc(op_sig_32,
    locals=dict(b=uint32,occ=int32,sps=uint32,me_offdiag=float64,me_diag=float64), )
def op(op_struct_ptr,op_str,site_ind,N,args):
    # using struct pointer to pass op_struct_ptr back to C++ see numba Records
    op_struct = carray(op_struct_ptr,1)[0]
    err = 0
    sps=args[0]
    me_offdiag=1.0;
    me_diag=1.0;
    #
    site_ind = N - site_ind - 1 # convention for QuSpin for mapping from bits to sites.
    b = sps**site_ind
    occ = (op_struct.state//b)%sps # occupation
    #
    #opstr_left,opstr_right=op_str.split("|",1)
    # spin-part operators :
    if site_ind==0 : # the zero-th site contains the spin of the atom
#         if opstr_left==120: # "x" is integer value 120 = ord("x")
#             op_struct.state ^= b

#         elif opstr_left==121: # "y" is integer value 120 = ord("y")
#             op_struct.state ^= b
#             op_struct.matrix_ele *= 1.0j*s

        if op_str==49: # "1" 
            if (occ > 0): op_struct.matrix_ele = 0
            else: op_struct.state += (b if occ==0 else 0)  # create spin

        elif op_str==48: # "0" 
            if (occ > 0): op_struct.state -= (b if occ>0 else 0) # destroy spin
            else: op_struct.matrix_ele = 0

        elif op_str==122: # "z" is integer value 120 = ord("z")
            op_struct.matrix_ele *= (-1 if occ==0 else 1)

        # elif op_str==110: # "n" is integer value 110 = ord("n")
        #     op_struct.matrix_ele *= n

        elif op_str==73: # "I" is integer value 73 = ord("I")
            pass

        else:
            op_struct.matrix_ele = 0
            err = -1   
    else :
        # boson-part operators :
        if op_str==43: # "+" is integer value 43 = ord("+")
            me_offdiag *= (occ+1)%sps
            op_struct.state += (b if (occ+1)<sps else 0) 

        elif op_str==45: # "-" is integer value 45 = ord("-")
            me_offdiag *= occ;
            op_struct.state -= (b if occ>0 else 0)

        elif op_str==110: # "n" is integer value 110 = ord("n")
            me_diag *= occ

        elif op_str==73: # "I" is integer value 73 = ord("I")
            pass

        else:
            me_diag = 0.0
            err = -1
        #
        op_struct.matrix_ele *= me_diag*np.sqrt(me_offdiag)
    #
    return err
#
op_args=np.array([sps],dtype=np.uint32)

In [7]:
@cfunc(next_state_sig_32,
    locals=dict(t=uint32,i=int32,j=int32,n=int32,sps=uint32,b1=int32,b2=int32,l=int32,n_left=int32), )
def next_state(s,counter,N,args):
    """ implements particle number conservation. Particle number set by initial state, cf `get_s0_pcon()` below. """
    t = s;
    sps=args[1]
    n=0 # running total of number of particles
    for i in range(N): # loop over lattices sites
        b1 = (t//args[i])%sps # get occupation at site i
        if b1>0: # if there is a boson
            n += b1 
            b2 = (t/args[i+1])%sps # get occupation at site ahead
            if (b2<(sps-1)  and i<N-2) or (b2==0 and i==N-2)  : # if I can move a boson to this site
                n -= 1 # decrease one from the running total
                t -= args[i] # remove one boson from site i 
                t += args[i+1] # add one boson to site i+1
                if n>0: # if any bosons left
                    # so far: moved one boson forward; 
                    # now: take rest of bosons and fill first l sites with maximum occupation 
                    # to keep lexigraphic order
                    l = n//(sps-1) # how many sites can be fully occupied with n bosons
                    n_left = n%(sps-1) # leftover of particles on not maximally occupied sites
                    for j in range(i+1):
                        t -= (t//args[j])%sps * args[j];
                        if j<l: # fill in with maximal occupation
                            t += (sps-1)*args[j]
                        elif j==l: # fill with leftover
                            t += n_left*args[j]
                break # stop loop
    return t
next_state_args=np.array([sps**i for i in range(N)],dtype=np.uint32)

In [8]:
def get_s0_pcon(N,Np):
    l = Np//(sps-1)
    s  = sum((sps-1) * sps**i for i in range(l))
    s += (Np%(sps-1)) * sps**l
    return s

In [9]:
# python function to calculate the size of the particle-conserved basis, i.e. 
# BEFORE applying pre_check_state and symmetry maps
def get_Ns_pcon(N,Np):
    Ns=0
    # count states with spin down (0)
    for r in range((Np)//sps+1):
        r_2 = Np - r*sps
        if r % 2 == 0:
            Ns +=  comb(N-1,r,exact=True) * comb(N-1 + r_2 - 1,r_2,exact=True)
        else:
            Ns += -comb(N-1,r,exact=True) * comb(N-1 + r_2 - 1,r_2,exact=True)
    # count states with spin up (1)
    for r in range((Np-1)//sps+1):
        r_2 = Np - 1 - r*sps
        if r % 2 == 0:
            Ns +=  comb(N-1,r,exact=True) * comb(N-1 + r_2 - 1,r_2,exact=True)
        else:
            Ns += -comb(N-1,r,exact=True) * comb(N-1 + r_2 - 1,r_2,exact=True)
    return Ns

In [10]:
# ######  define symmetry maps
# #
# @cfunc(map_sig_32,
#     locals=dict(shift=uint32,out=uint32,sps=uint32,i=int32,j=int32,) )
# def translation(x,N,sign_ptr,args):
#     """ works for all system sizes N. """
#     out = 0
#     shift = args[0]
#     sps = args[1]
#     for i in range(N):
#         j = (i+shift+N)%N
#         out += ( x%sps ) * sps**j
#         x //= sps
#     #
#     return out
# T_args=np.array([1,sps],dtype=np.uint32)
# #
# @cfunc(map_sig_32,
#     locals=dict(out=uint32,sps=uint32,i=int32,j=int32) )
# def parity(x,N,sign_ptr,args):
#     """ works for all system sizes N. """
#     out = 0
#     sps = args[0]
#     for i in range(N):
#         j = (N-1) - i
#         out += ( x%sps ) * (sps**j)
#         x //= sps
#     #
#     return out
# P_args=np.array([sps],dtype=np.uint32)

In [11]:
######  define function to count particles in bit representation
#
@cfunc(count_particles_sig_32,
    locals=dict(s=uint32,))
def count_particles(x,p_number_ptr,args):
    """ Counts number of particles/spin-ups in a state stored in integer representation for up to N=32 sites """
    #
    s = x # integer x cannot be changed
    for i in range(args[0]):
        p_number_ptr[0] += s%args[1]
        s /= args[1]
n_sectors=1 # number of particle sectors
count_particles_args=np.array([N,sps],dtype=np.int32)

# Construct user_basis 

In [12]:
# define maps dict
#maps = dict(T_block=(translation,N,0,T_args),P_block=(parity,2,0,P_args), ) 

In [13]:
# define particle conservation and op dicts
pcon_dict = dict(Np=Np,next_state=next_state,next_state_args=next_state_args,
                 get_Ns_pcon=get_Ns_pcon,get_s0_pcon=get_s0_pcon,
                 count_particles=count_particles,count_particles_args=count_particles_args,n_sectors=n_sectors)
op_dict = dict(op=op,op_args=op_args)

In [14]:
# create user basiss
basis = user_basis(np.uint32,N,op_dict,allowed_ops=set("+-nI"),sps=sps,pcon_dict=pcon_dict)

In [15]:
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
      0.         |1 2 1 0>           48  
      1.         |1 2 0 1>           46  
      2.         |1 1 2 0>           42  
      3.         |1 1 1 1>           40  
      4.         |1 1 0 2>           38  
      5.         |1 0 2 1>           34  
      6.         |1 0 1 2>           32  
      7.         |0 2 2 0>           24  
      8.         |0 2 1 1>           22  
      9.         |0 2 0 2>           20  
     10.         |0 1 2 1>           16  
     11.         |0 1 1 2>           14  
     12.         |0 0 2 2>            8  


# Create Hamiltonians 

In [16]:
N_lev=sps # maximum photon occupation 
N_ph_avg=N_lev/2 # mean number of photons in initial coherent state
Omega=1.0 # drive frequency
A=0.5 # spin-photon coupling strength (drive amplitude)
Delta=1.0 # difference between atom energy levels
N_osc=Np-1

In [17]:
# define operator site-coupling lists
ph_energy=[[Omega,i] for i in range(N_osc)] # photon energy
at_energy=[[Delta,0]] # atom energy, 0 because the "lattice" is single-site
absorb=[[A/(2.0*np.sqrt(N_ph_avg)),0,i] for i in range(N_osc)] # absorption term
emit=[[A/(2.0*np.sqrt(N_ph_avg)),0,i] for i in range(N_osc)] # emission term

In [21]:
static_rotating_wave=[["n",ph_energy],["1-",absorb],["0+",emit],["z",at_energy]]
dynamic=[]

In [22]:
no_checks=dict(check_symm=False, check_pcon=False, check_herm=False)
H=hamiltonian(static_rotating_wave,[],basis=basis,dtype=np.float64,**no_checks)

ValueError: unrecognized characters {'1'} in operator string.

In [ ]:
print(H.toarray())